In [1]:
import time
import requests
import pandas as pd
from tqdm import tqdm 
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
browser = webdriver.Chrome()
browser.get('https://stockmages.netlify.app')

In [3]:
#Infinite Scroll to reach the bottom 
for i in tqdm(range(0, 200000, 1000)):
    browser.execute_script("window.scrollTo(0,"  + str(i) + ")")
    time.sleep(.1)

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:21<00:00,  9.32it/s]


In [4]:
# Scraping the Page
soup = BeautifulSoup(browser.page_source, 'html.parser')

In [5]:
#Scraping the Image Details
data = []
for sp in tqdm(soup.find_all('div', class_ = 'container')):
    img_link = sp.find('img').get('src')
    tags     = sp.find('span', class_ = 'tag-color').text[7:].strip()
    likes    = int(sp.find('div', class_ = 'likes-comments').find_all('span')[0].text.strip()[:-6])
    comments = int(sp.find('div', class_ = 'likes-comments').find_all('span')[1].text.strip()[:-9]) 
    
    data.append([img_link, tags, likes, comments])

100%|█████████████████████████████████████████████████████████████████████████████| 662/662 [00:00<00:00, 11853.22it/s]


In [6]:
#Creating a dataframe and saving it
df = pd.DataFrame(data, columns = ['img_link', 'tags', 'likes', 'comments'])
df.head()

,img_link,tags,likes,comments
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"Clouds, Sky, Atmosphere, Blue Sky",196,55
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"Bird, Ornithology, Hummingbird",76,20
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"Sea, Rainbow, Rainfall, Subtropical",282,106
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"Cherry Blossoms, Road, Japan, Sakura",42,11
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"Cape Marguerite, Flower, Plant",39,15


In [7]:
def download(link, img_path):
    res = requests.get(link)
    with open(img_path, 'wb') as file:
        file.write(res.content)

In [9]:
#Downloading all Images.
paths = []
for link in tqdm(df['img_link']):
    img_path = 'Imgs/' + link.split('/')[-1]
    download(link, img_path) # В РФ нужен VPN
    paths.append(img_path)
df['path'] = paths
df.head()

100%|████████████████████████████████████████████████████████████████████████████████| 662/662 [07:05<00:00,  1.56it/s]


,img_link,tags,likes,comments,path
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"Clouds, Sky, Atmosphere, Blue Sky",196,55,Imgs/clouds-7050884__480.jpg
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"Bird, Ornithology, Hummingbird",76,20,Imgs/bird-7117346__340.jpg
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"Sea, Rainbow, Rainfall, Subtropical",282,106,Imgs/sea-7039471__340.jpg
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"Cherry Blossoms, Road, Japan, Sakura",42,11,Imgs/cherry-blossoms-7110279__340.jpg
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"Cape Marguerite, Flower, Plant",39,15,Imgs/cape-marguerite-7121992__340.jpg


In [10]:
df.to_csv('images.csv', index = False)